In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/news-category-dataset/News_Category_Dataset_v3.json


In [21]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
import torch
from sklearn.metrics import classification_report


In [8]:
df=pd.read_json("/kaggle/input/news-category-dataset/News_Category_Dataset_v3.json",lines=True)

In [9]:
# Load the stopwords and initialize the stemmer and lemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    # Remove punctuations
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stopwords and stem or lemmatize the words
    words = [stemmer.stem(word) for word in tokens if word not in stop_words]
    # words = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    # Join the words back into a string
    processed_text = ' '.join(words)
    return processed_text

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [10]:
# Preprocess the 'headline' column
df['headline'] = df['headline'].apply(preprocess_text)

# Preprocess the 'short_description' column
df['short_description'] = df['short_description'].apply(preprocess_text)

In [12]:
top_categories = ['POLITICS', 'WORLD NEWS', 'ENTERTAINMENT', 'SPORTS', 'BUSINESS', 'TECH']

# Filter out rows with categories not in the top 6
df = df[df['category'].isin(top_categories)]


In [13]:
df.shape

(69436, 6)

In [14]:
# Load the pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Split the dataset into train and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(df['headline'] + ' ' + df['short_description'], df['category'], test_size=0.2, random_state=42)

# Tokenize the train and test texts
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True)
test_encodings = tokenizer(test_texts.tolist(), truncation=True, padding=True)


In [16]:
# Convert the labels to numerical values
labels_map = {label: i for i, label in enumerate(train_labels.unique())}
train_labels = [labels_map[label] for label in train_labels.tolist()]
test_labels = [labels_map[label] for label in test_labels.tolist()]

# Convert the train and test data to PyTorch tensors
train_dataset = torch.utils.data.TensorDataset(torch.tensor(train_encodings['input_ids']),
                                               torch.tensor(train_encodings['attention_mask']),
                                               torch.tensor(train_labels))
test_dataset = torch.utils.data.TensorDataset(torch.tensor(test_encodings['input_ids']),
                                              torch.tensor(test_encodings['attention_mask']),
                                              torch.tensor(test_labels))

# Load the pre-trained BERT model and modify the last layer for classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(labels_map))


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)

In [19]:
# Train the model
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
optim = torch.optim.AdamW(model.parameters(), lr=5e-5)
epochs = 3
for epoch in range(epochs):
    for batch in train_loader:
        optim.zero_grad()
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optim.step()

In [20]:
# Evaluate the model on the test set
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        _, predicted = torch.max(outputs[0], 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
accuracy = correct / total
print('Test Accuracy: {:.2f}'.format(accuracy))

Test Accuracy: 0.88


In [22]:
# Get the predictions for the test set
model.eval()
y_pred = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        _, predicted = torch.max(outputs[0], 1)
        y_pred.extend(predicted.cpu().numpy())

# Get the actual labels for the test set
y_true = test_labels

# Print the classification report
target_names = list(labels_map.keys())
print(classification_report(y_true, y_pred, target_names=target_names))

               precision    recall  f1-score   support

     POLITICS       0.93      0.92      0.92      7095
ENTERTAINMENT       0.90      0.92      0.91      3537
       SPORTS       0.83      0.89      0.85       986
     BUSINESS       0.75      0.68      0.72      1209
   WORLD NEWS       0.71      0.79      0.75       642
         TECH       0.75      0.65      0.70       419

     accuracy                           0.88     13888
    macro avg       0.81      0.81      0.81     13888
 weighted avg       0.88      0.88      0.88     13888



In [25]:
def predict_category(article_text):
    # preprocess the article text
    preprocessed_text = preprocess_text(article_text)
    
    # tokenize the text
    input_ids = tokenizer.encode(preprocessed_text, truncation=True, padding=True, return_tensors='pt')
    
    # make a prediction
    with torch.no_grad():
        outputs = model(input_ids.to(device), attention_mask=None)
        _, predicted = torch.max(outputs[0], 1)
    
    # get the predicted category label
    label_map = {v: k for k, v in labels_map.items()}
    predicted_label = label_map[predicted.item()]
    
    return predicted_label

In [28]:
article = "Apple unveils new iPhone"
predicted_category = predict_category(article)
print(predicted_category)


TECH


In [29]:
article = "Donald Trump announces presidential campaign"
predicted_category = predict_category(article)
print(predicted_category)


POLITICS


In [30]:
article = "Serena Williams wins Wimbledon"
predicted_category = predict_category(article)
print(predicted_category)


SPORTS
